# Генерация последовательности перехода от картинки к картинке
Используются веса, созданные нейросетью (learn.ipynb), обученной на разных картинках.      
Веса, обученые на одной картинке изменяются, что бы через заданное количество шагов        
соответствовать весам, обученным на другой картинке.        
На каждом шаге происходит сохранение изображения в **.png** файл указаного размера.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import scipy.special
import imageio
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import random

#### дополнительные функции

In [ ]:
def get_picture(model0,width,height,name):
    """функция получение и сохранение картинки от нейросети"""
    inp = np.zeros((width*height, 2)) 
    n = 0
    for x in range(0,width):
        for y in range(0,height):
            inp[n][0] = 2*x/width - 1
            inp[n][1] = 2*y/height- 1
            n += 1

    y_proba = model0.predict(inp)
    n = 0
    for x in range(0,width): #
        for y in range(0,height): #
            imout[y,x] = (y_proba[n]+1)*255/2
            n += 1  
        
        
def doit(stepsCount,firstModel,secondModel):
    global startFileName
    global imout
    model0 = keras.models.load_model(firstModel)
    model1 = keras.models.load_model(firstModel)
    model2 = keras.models.load_model(secondModel)
    q=len(model0.layers)
    for currentStep in range(0,stepsCount):
        for n in range(0,q):
            w1,b1 = model1.layers[n].get_weights()
            w2,b2 = model2.layers[n].get_weights()
            ws = w1+currentStep*(w2-w1)/(stepsCount-1)
            bs = b1+currentStep*(b2-b1)/(stepsCount-1)
            model0.layers[n].set_weights([ws,bs])
        
        get_picture(model0,width,height,startFileName)
        imageio.imwrite(folder+'/'+str(startFileName)+'.png', imout[:, :, :].astype(np.uint8))
        startFileName += 1
        

### Параметры
- Ширина и высота изображений на выходе
- Папка, где всё это нужно сохранить

In [ ]:
width = 300
height = 270
folder = 'result'  # имя папки, где будут сохраняться скриншоты (создаётся вручную)

# здесь будем формировать картинку
imout = np.zeros(( height, width , 3))


### Финальная часть   
**startFileName** - число, из которого сгенерируется имя начального файла
Первым агрументом у функции **doit()** указываем количество кадров, которое будет длится преобразование.      
Затем адрес нейросети с которой начнется генерироватся последовательность    
и адрес нейросети на которой последовательность завершиться

In [ ]:
print(f'Ждите. идёт генерация, за результатом можно следить в папке {folder}')
startFileName = 0

doit(30,"pic/01.h5", "pic/02.h5")
doit(30,"pic/02.h5", "pic/03.h5")
doit(30,"pic/03.h5", "pic/04.h5")
doit(30,"pic/04.h5", "pic/05.h5")
doit(30,"pic/05.h5", "pic/00.h5")
print('всё')